#### 备用命令行

In [15]:
!pip install tensorflow-gpu==1.13.1

    100% |████████████████████████████████| 345.2MB 82.7MB/s eta 0:00:01     | 37.0MB 86.9MB/s eta 0:00:04         | 56.4MB 82.2MB/s eta 0:00:04███                          | 65.9MB 47.7MB/s eta 0:00:068MB 84.0MB/s eta 0:00:04 93.4MB 63.5MB/s eta 0:00:04��████▏                      | 98.9MB 64.4MB/s eta 0:00:04�███████                    | 130.4MB 102.8MB/s eta 0:00:0384.5MB/s eta 0:00:03███▋                  | 146.4MB 98.6MB/s eta 0:00:03   44% |██████████████▍                 | 154.9MB 84.0MB/s eta 0:00:03   45% |██████████████▋                 | 158.0MB 67.8MB/s eta 0:00:03   | 165.7MB 86.1MB/s eta 0:00:03   52% |████████████████▊               | 180.3MB 56.0MB/s eta 0:00:03��████████               | 184.1MB 72.4MB/s eta 0:00:03:02[K    55% |█████████████████▊              | 190.7MB 85.3MB/s eta 0:00:02[K    56% |██████████████████              | 195.0MB 51.1MB/s eta 0:00:03�███████▋             | 200.8MB 78.3MB/s eta 0:00:02 |████████████████████▉           | 224.8MB 85.1MB/s eta 0

In [6]:
!rm -f example.*

In [3]:
!rm -rf model2

In [14]:
!cd model; dir

/bin/sh: line 0: cd: model: No such file or directory
data  data.py  elmo.py	model1	__pycache__


In [4]:
!dir 

data  data.py  elmo.py	model3	__pycache__


#### 华为云搬运文件至当前notebook dir

In [34]:
import moxing as mox

# mox.file.copy_parallel('obs://class-1275-41780/Lab-1799/modelarts22888991/data', '/home/ma-user/work/data')
mox.file.copy_parallel('obs://class-1275-41780/Lab-1799/modelarts22888991/model3', '/home/ma-user/work/model3')

### Section 1:  使用预训练模型进行预测

#### step 1: 引入包，参数设置，类加载

> 需要将emlo.py 和 data.py 两个文件放在同目录下                  
> 需要将data文件夹放在同目录下

In [14]:
import tensorflow as tf
from tensorflow.contrib import seq2seq
from elmo import ELMo
from data import NERData
import os

total_epoch = 5000
hidden_size = 200
vocab_size = 5000
max_length = 128
entity_class = 7

batch_size = 1    # 跟训练的区别

ner = NERData(batch_size, max_length)
elmo = ELMo(batch_size, hidden_size, vocab_size)

成功加载语料./data/example.train, 语料数量18404
成功加载语料./data/example.test, 语料数量4558
成功加载语料./data/example.dev, 语料数量4384


#### step 2: 顶层softmax layer函数定义

In [6]:
def network(X):
#     with tf.variable_scope(reuse=tf.AUTO_REUSE):
        w = tf.get_variable("fcn_w", [1, hidden_size, entity_class + 1])
        b = tf.get_variable("fcn_b", [entity_class + 1])
        # 这里输出维度用entity_class + 1而不是entity_class，因为输出里除了7类实体，还有一类用来表示每个句子补齐的<PAD>位
        w_tile = tf.tile(w, [batch_size, 1, 1])

        logists = tf.nn.softmax(tf.nn.xw_plus_b(X, w_tile, b), name="logists")
        return logists

#### step 3: 载入模型

In [7]:
X = tf.placeholder(shape=[batch_size, max_length], dtype=tf.int32, name="X")
length = tf.placeholder(shape=[batch_size], dtype=tf.int32, name="length")
dropout = tf.placeholder(shape=[], dtype=tf.float32, name="dropout")

elmo_vector = elmo.elmo(X, length, dropout)
logists = network(elmo_vector)

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
config = tf.ConfigProto(gpu_options=gpu_options)
sess = tf.Session(config=config)

sess.run(tf.global_variables_initializer())
model_dir = "./model3"
saver = tf.train.Saver()
check_point = tf.train.get_checkpoint_state(model_dir)
saver.restore(sess, check_point.model_checkpoint_path)




Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model3/model3-14301


#### step 4: 查看elmo各层的权重

In [8]:
_X, _length, _targets = ner.get_dev_data(1)
fd = {X: _X, length: _length, dropout: 1.}
weights = sess.run(tf.nn.softmax(elmo.weights), feed_dict=fd)
print("embedding层:", weights[0])
print("forward_1层:", weights[1])
print("forward_2层:", weights[2])
print("backward_1层:", weights[3])
print("backward_2层:", weights[4])

embedding层: 0.017066773
forward_1层: 0.15887623
forward_2层: 0.3802379
backward_1层: 0.23893026
backward_2层: 0.20488887


#### step 5: 用来自不同领域的句子进行测试

In [10]:
s = "中国石油大学（北京）克拉玛依校区设有石油工程、地质、储运等石油大学传统优势专业，并开展面向中亚地区的留学生教育。"
_X, _length = ner.sentence_encode(s)
fd = {X: _X, length: _length, dropout: 1.}
result = sess.run(logists, feed_dict=fd)
result_entities = ner.entities_decode(result[0].argmax(axis=1))
list(zip(s, result_entities))[:_length[0]][:len(s)]

[('中', 'B-ORG'),
 ('国', 'I-ORG'),
 ('石', 'I-ORG'),
 ('油', 'I-ORG'),
 ('大', 'I-ORG'),
 ('学', 'I-ORG'),
 ('（', 'O'),
 ('北', 'B-LOC'),
 ('京', 'I-LOC'),
 ('）', 'I-LOC'),
 ('克', 'I-LOC'),
 ('拉', 'I-LOC'),
 ('玛', 'I-PER'),
 ('依', 'I-ORG'),
 ('校', 'I-ORG'),
 ('区', 'O'),
 ('设', 'O'),
 ('有', 'O'),
 ('石', 'O'),
 ('油', 'O'),
 ('工', 'O'),
 ('程', 'O'),
 ('、', 'O'),
 ('地', 'O'),
 ('质', 'O'),
 ('、', 'O'),
 ('储', 'O'),
 ('运', 'O'),
 ('等', 'O'),
 ('石', 'O'),
 ('油', 'O'),
 ('大', 'O'),
 ('学', 'O'),
 ('传', 'O'),
 ('统', 'O'),
 ('优', 'O'),
 ('势', 'O'),
 ('专', 'O'),
 ('业', 'O'),
 ('，', 'O'),
 ('并', 'O'),
 ('开', 'O'),
 ('展', 'O'),
 ('面', 'O'),
 ('向', 'O'),
 ('中', 'B-LOC'),
 ('亚', 'I-LOC'),
 ('地', 'O'),
 ('区', 'O'),
 ('的', 'O'),
 ('留', 'O'),
 ('学', 'O'),
 ('生', 'O'),
 ('教', 'O'),
 ('育', 'O'),
 ('。', 'O')]

In [11]:
s = "李克强来到位于江西省赣州市于都县的梓山镇潭头村看望慰问群众"
_X, _length = ner.sentence_encode(s)
fd = {X: _X, length: _length, dropout: 1.}
result = sess.run(logists, feed_dict=fd)
result_entities = ner.entities_decode(result[0].argmax(axis=1))
list(zip(s, result_entities))[:_length[0]][:len(s)]

[('李', 'B-PER'),
 ('克', 'I-PER'),
 ('强', 'I-PER'),
 ('来', 'O'),
 ('到', 'O'),
 ('位', 'O'),
 ('于', 'O'),
 ('江', 'B-LOC'),
 ('西', 'I-LOC'),
 ('省', 'I-LOC'),
 ('赣', 'B-LOC'),
 ('州', 'I-LOC'),
 ('市', 'I-LOC'),
 ('于', 'O'),
 ('都', 'O'),
 ('县', 'B-LOC'),
 ('的', 'O'),
 ('梓', 'B-LOC'),
 ('山', 'I-LOC'),
 ('镇', 'I-LOC'),
 ('潭', 'I-LOC'),
 ('头', 'I-LOC'),
 ('村', 'I-LOC'),
 ('看', 'O'),
 ('望', 'O'),
 ('慰', 'O'),
 ('问', 'O'),
 ('群', 'O'),
 ('众', 'O')]

In [7]:
s = "多名白宫官员对媒体表示，美国总统特朗普不准备续签即将到期的美俄《新削减战略武器条约》，而是想推动达成一项包括中国在内的新条约"
_X, _length = ner.sentence_encode(s)
fd = {X: _X, length: _length, dropout: 1.}
result = sess.run(logists, feed_dict=fd)
result_entities = ner.entities_decode(result[0].argmax(axis=1))
list(zip(s, result_entities))[:_length[0]][:len(s)]

[('多', 'O'),
 ('名', 'O'),
 ('白', 'B-ORG'),
 ('宫', 'I-ORG'),
 ('官', 'O'),
 ('员', 'O'),
 ('对', 'O'),
 ('媒', 'O'),
 ('体', 'O'),
 ('表', 'O'),
 ('示', 'O'),
 ('，', 'O'),
 ('美', 'B-LOC'),
 ('国', 'I-LOC'),
 ('总', 'O'),
 ('统', 'O'),
 ('特', 'B-PER'),
 ('朗', 'I-LOC'),
 ('普', 'I-PER'),
 ('不', 'O'),
 ('准', 'O'),
 ('备', 'O'),
 ('续', 'O'),
 ('签', 'O'),
 ('即', 'O'),
 ('将', 'O'),
 ('到', 'O'),
 ('期', 'O'),
 ('的', 'O'),
 ('美', 'B-LOC'),
 ('俄', 'B-LOC'),
 ('《', 'O'),
 ('新', 'O'),
 ('削', 'O'),
 ('减', 'O'),
 ('战', 'O'),
 ('略', 'O'),
 ('武', 'O'),
 ('器', 'O'),
 ('条', 'O'),
 ('约', 'O'),
 ('》', 'O'),
 ('，', 'O'),
 ('而', 'O'),
 ('是', 'O'),
 ('想', 'O'),
 ('推', 'O'),
 ('动', 'O'),
 ('达', 'O'),
 ('成', 'O'),
 ('一', 'O'),
 ('项', 'O'),
 ('包', 'O'),
 ('括', 'O'),
 ('中', 'B-LOC'),
 ('国', 'I-LOC'),
 ('在', 'O'),
 ('内', 'O'),
 ('的', 'O'),
 ('新', 'O'),
 ('条', 'O'),
 ('约', 'O')]

In [12]:
s = "小罗伯特·唐尼专门为漫威总裁凯文费奇颁发特别大奖，他在介绍凯文费奇的时候说道“我要感谢凯文，他在我的低谷期认可我“"
_X, _length = ner.sentence_encode(s)
fd = {X: _X, length: _length, dropout: 1.}
result = sess.run(logists, feed_dict=fd)
result_entities = ner.entities_decode(result[0].argmax(axis=1))
list(zip(s, result_entities))[:_length[0]][:len(s)]

[('小', 'B-PER'),
 ('罗', 'B-PER'),
 ('伯', 'I-PER'),
 ('特', 'I-PER'),
 ('·', 'I-PER'),
 ('唐', 'I-PER'),
 ('尼', 'I-PER'),
 ('专', 'O'),
 ('门', 'O'),
 ('为', 'O'),
 ('漫', 'O'),
 ('威', 'O'),
 ('总', 'O'),
 ('裁', 'O'),
 ('凯', 'B-PER'),
 ('文', 'I-PER'),
 ('费', 'I-PER'),
 ('奇', 'I-PER'),
 ('颁', 'O'),
 ('发', 'O'),
 ('特', 'O'),
 ('别', 'O'),
 ('大', 'O'),
 ('奖', 'O'),
 ('，', 'O'),
 ('他', 'O'),
 ('在', 'O'),
 ('介', 'O'),
 ('绍', 'O'),
 ('凯', 'B-PER'),
 ('文', 'I-PER'),
 ('费', 'I-PER'),
 ('奇', 'I-PER'),
 ('的', 'O'),
 ('时', 'O'),
 ('候', 'O'),
 ('说', 'O'),
 ('道', 'O'),
 ('“', 'O'),
 ('我', 'O'),
 ('要', 'O'),
 ('感', 'O'),
 ('谢', 'O'),
 ('凯', 'O'),
 ('文', 'O'),
 ('，', 'O'),
 ('他', 'O'),
 ('在', 'O'),
 ('我', 'O'),
 ('的', 'O'),
 ('低', 'O'),
 ('谷', 'O'),
 ('期', 'O'),
 ('认', 'O'),
 ('可', 'O'),
 ('我', 'O'),
 ('“', 'O')]

In [13]:
s = "马苏、周冬雨、林更新、霍建华都被拍到出现在周杰伦的演唱会上，此外，林俊杰和陈奕迅等超级巨星也都作为演唱会嘉宾出现在演唱会上"
_X, _length = ner.sentence_encode(s)
fd = {X: _X, length: _length, dropout: 1.}
result = sess.run(logists, feed_dict=fd)
result_entities = ner.entities_decode(result[0].argmax(axis=1))
list(zip(s, result_entities))[:_length[0]][:len(s)]

[('马', 'B-PER'),
 ('苏', 'I-PER'),
 ('、', 'O'),
 ('周', 'B-PER'),
 ('冬', 'I-PER'),
 ('雨', 'I-PER'),
 ('、', 'O'),
 ('林', 'B-PER'),
 ('更', 'I-PER'),
 ('新', 'I-PER'),
 ('、', 'O'),
 ('霍', 'B-PER'),
 ('建', 'I-PER'),
 ('华', 'I-PER'),
 ('都', 'O'),
 ('被', 'O'),
 ('拍', 'O'),
 ('到', 'O'),
 ('出', 'O'),
 ('现', 'O'),
 ('在', 'O'),
 ('周', 'B-PER'),
 ('杰', 'I-PER'),
 ('伦', 'I-PER'),
 ('的', 'O'),
 ('演', 'O'),
 ('唱', 'O'),
 ('会', 'O'),
 ('上', 'O'),
 ('，', 'O'),
 ('此', 'O'),
 ('外', 'O'),
 ('，', 'O'),
 ('林', 'B-PER'),
 ('俊', 'I-PER'),
 ('杰', 'I-PER'),
 ('和', 'O'),
 ('陈', 'B-PER'),
 ('奕', 'I-PER'),
 ('迅', 'I-PER'),
 ('等', 'O'),
 ('超', 'O'),
 ('级', 'O'),
 ('巨', 'O'),
 ('星', 'O'),
 ('也', 'O'),
 ('都', 'O'),
 ('作', 'O'),
 ('为', 'O'),
 ('演', 'O'),
 ('唱', 'O'),
 ('会', 'O'),
 ('嘉', 'O'),
 ('宾', 'O'),
 ('出', 'O'),
 ('现', 'O'),
 ('在', 'O'),
 ('演', 'O'),
 ('唱', 'O'),
 ('会', 'O'),
 ('上', 'O')]

### Section 2: train a model [1. from scratch, 2. incremental training]

In [1]:
import tensorflow as tf
from tensorflow.contrib import seq2seq
from elmo import ELMo
from data import NERData
import os

total_epoch = 100
hidden_size = 200
vocab_size = 5000
max_length = 128
entity_class = 7

lr = 1e-3
batch_size = 128

ner = NERData(batch_size, max_length)
elmo = ELMo(batch_size, hidden_size, vocab_size)


def network(X):
    w = tf.get_variable("fcn_w", [1, hidden_size, entity_class + 1])
    b = tf.get_variable("fcn_b", [entity_class + 1])
    # 这里输出维度用entity_class + 1而不是entity_class，因为输出里除了7类实体，还有一类用来表示每个句子补齐的<PAD>位
    w_tile = tf.tile(w, [batch_size, 1, 1])

    logists = tf.nn.softmax(tf.nn.xw_plus_b(X, w_tile, b), name="logists")
    return logists


def train():
    X = tf.placeholder(shape=[batch_size, max_length], dtype=tf.int32, name="X")
    length = tf.placeholder(shape=[batch_size], dtype=tf.int32, name="length")
    targets = tf.placeholder(shape=[batch_size, max_length], dtype=tf.int32, name="targets")
    weights = tf.placeholder(shape=[batch_size, max_length], dtype=tf.float32, name="weights")
    dropout = tf.placeholder(shape=[], dtype=tf.float32, name="dropout")

    elmo_vector = elmo.elmo(X, length, dropout)
    logists = network(elmo_vector)

    seq_loss = seq2seq.sequence_loss(logists, targets, weights)
    # optimizer = tf.train.AdamOptimizer(lr).minimize(seq_loss)

    trainableVariables = tf.trainable_variables()
    optimizer = tf.train.AdamOptimizer(lr)
    grads, a = tf.clip_by_global_norm(tf.gradients(seq_loss, trainableVariables), 5)
    train_op = optimizer.apply_gradients(zip(grads, trainableVariables))

    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
    config = tf.ConfigProto(gpu_options=gpu_options)

    with tf.Session(config=config) as sess:
        sess.run(tf.global_variables_initializer())
        model_dir = "model1"
        saver = tf.train.Saver(max_to_keep=10)
        if not os.path.exists(model_dir):   # 检查./model路径是否存在
            os.mkdir(model_dir)             # 不存在就创建路径
            print("create the directory: %s" % model_dir)
        check_point = tf.train.get_checkpoint_state(model_dir)
        if check_point and check_point.model_checkpoint_path:
            saver.restore(sess, check_point.model_checkpoint_path)
            print("restored %s" % check_point.model_checkpoint_path)
        else:
            print("no checkpoint found!")

        step = 0
        total_loss = 0
        while ner.epoch < total_epoch:
            _X, _length, _targets, _weights = ner.get_train_batch()
            fd = {X: _X, length: _length, targets: _targets, weights: _weights, dropout: .75}
            _, l = sess.run([train_op, seq_loss], feed_dict=fd)
            total_loss += l
            if step % 100 == 0:
                print("epoch:", ner.epoch, "step:", step, "loss:", total_loss / 100)
                total_loss = 0

            if step % 1000 == 0:
                _X, _length, _targets, _weights = ner.get_test_batch()
                fd = {X: _X, length: _length, targets: _targets, weights: _weights, dropout: 1.}
                l, result = sess.run([seq_loss, logists], feed_dict=fd)
                result = result.argmax(axis=2)
                ner_num = 0
                for i in range(batch_size):
                    for j in range(_length[i]):
                        if result[i, j] != 1:
                            ner_num += 1
                print("平均每句有标注的实体数:", ner_num / batch_size)
                print("test_loss:", l)

#             if step % 10000 == 0:
            step += 1
        saver.save(sess, model_dir, global_step=step)  # 保存模型
        print("saving...")

            
def incremental_train():
    X = tf.placeholder(shape=[batch_size, max_length], dtype=tf.int32, name="X")
    length = tf.placeholder(shape=[batch_size], dtype=tf.int32, name="length")
    targets = tf.placeholder(shape=[batch_size, max_length], dtype=tf.int32, name="targets")
    weights = tf.placeholder(shape=[batch_size, max_length], dtype=tf.float32, name="weights")
    dropout = tf.placeholder(shape=[], dtype=tf.float32, name="dropout")
#     dropout = tf.placeholder(shape=[], dtype=tf.float32, name="dropout")

    elmo_vector = elmo.elmo(X, length, dropout)
    logists = network(elmo_vector)

    seq_loss = seq2seq.sequence_loss(logists, targets, weights)
    # optimizer = tf.train.AdamOptimizer(lr).minimize(seq_loss)

    trainableVariables = tf.trainable_variables()
    optimizer = tf.train.AdamOptimizer(lr)
    grads, a = tf.clip_by_global_norm(tf.gradients(seq_loss, trainableVariables), 5)
    train_op = optimizer.apply_gradients(zip(grads, trainableVariables))
    
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
    config = tf.ConfigProto(gpu_options=gpu_options)
    sess = tf.Session(config=config)

    sess.run(tf.global_variables_initializer())
    model_dir = "./model2"
    saver = tf.train.Saver()
    check_point = tf.train.get_checkpoint_state(model_dir)
    saver.restore(sess, check_point.model_checkpoint_path)

#     gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
#     config = tf.ConfigProto(gpu_options=gpu_options)

#     with tf.Session(config=config) as sess:
#         sess.run(tf.global_variables_initializer())
    model_dir = "./model3"
    saver = tf.train.Saver(max_to_keep=10)
    if not os.path.exists(model_dir):   # 检查./model路径是否存在
        os.mkdir(model_dir)             # 不存在就创建路径
        print("create the directory: %s" % model_dir)
    check_point = tf.train.get_checkpoint_state(model_dir)
    if check_point and check_point.model_checkpoint_path:
        saver.restore(sess,check_point.model_checkpoint_path)
        print("restored %s" % check_point.model_checkpoint_path)
    else:
        print("no checkpoint found!")

    step = 0
    total_loss = 0
    while ner.epoch < total_epoch:
        _X, _length, _targets, _weights = ner.get_train_batch()
        fd = {X: _X, length: _length, targets: _targets, weights: _weights, dropout: .75}
        _, l = sess.run([train_op, seq_loss], feed_dict=fd)
        total_loss += l
        if step % 100 == 0:
            print("epoch:", ner.epoch, "step:", step, "loss:", total_loss / 100)
            total_loss = 0

        if step % 1000 == 0:
            _X, _length, _targets, _weights = ner.get_test_batch()
            fd = {X: _X, length: _length, targets: _targets, weights: _weights, dropout: 1.}
            l, result = sess.run([seq_loss, logists], feed_dict=fd)
            result = result.argmax(axis=2)
            ner_num = 0
            for i in range(batch_size):
                for j in range(_length[i]):
                    if result[i, j] != 1:
                        ner_num += 1
            print("平均每句有标注的实体数:", ner_num / batch_size)
            print("test_loss:", l)

#             if step % 10000 == 0:
        step += 1
    saver.save(sess, model_dir, global_step=step)  # 保存模型
    print("saving...")    

    
# 如果想from scratch 则使用函数train()
# 如果想incrementally train 则使用函数incremental_train()
if __name__ == "__main__":
    incremental_train()
#     train()

成功加载语料./data/example.train, 语料数量18404
成功加载语料./data/example.test, 语料数量4558
成功加载语料./data/example.dev, 语料数量4384



Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model2/model2-1421
create the directory: ./model3
no checkpoint found!
epoch: 0 step: 0 loss: 0.01298594355583191
平均每句有标注的实体数: 0.0
test_loss: 1.3244393
epoch: 0 step: 100 loss: 1.3893569839000701
epoch:

epoch: 20 step: 2900 loss: 1.3872063338756562
epoch: 20 step: 3000 loss: 1.3946579587459564
平均每句有标注的实体数: 0.0
test_loss: 1.4042026
epoch: 21 step: 3100 loss: 1.38686643242836
epoch: 22 step: 3200 loss: 1.3861156606674194
epoch: 23 step: 3300 loss: 1.3882762265205384
epoch: 23 step: 3400 loss: 1.391923747062683
epoch: 24 step: 3500 loss: 1.3897809720039367
epoch: 25 step: 3600 loss: 1.388321019411087
epoch: 25 step: 3700 loss: 1.392163075208664
epoch: 26 step: 3800 loss: 1.3863803148269653
epoch: 27 step: 3900 loss: 1.3863626873493196
epoch: 27 step: 4000 loss: 1.3956758236885072
平均每句有标注的实体数: 0.0
test_loss: 1.4033378
epoch: 28 step: 4100 loss: 1.3859070456027984
epoch: 29 step: 4200 loss: 1.3860981273651123
epoch: 30 step: 4300 loss: 1.3890169215202333
epoch: 30 step: 4400 loss: 1.3918448448181153
epoch: 31 step: 4500 loss: 1.3895723795890809
epoch: 32 step: 4600 loss: 1.3882213723659516
epoch: 32 step: 4700 loss: 1.3921224665641785
epoch: 33 step: 4800 loss: 1.3859326136112213
epoch: 34

#### 搬运操作可忽略

In [7]:
!dir

data  data.py  elmo.py	model1	model2	model3	__pycache__


In [6]:
!mv checkpoint model3

In [8]:
import moxing as mox
mox.file.copy_parallel('/home/ma-user/work/model3', 'obs://class-1275-41780/Lab-1799/modelarts22888991/model3', )

INFO:root:Using MoXing-v1.15.1-3fc51aac
INFO:root:Using OBS-Python-SDK-3.1.2
